# Trabalhando com os Dados de Crítica de Jogos 

In [2]:
# Importando Pacotes
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy 
from sklearn import datasets, linear_model
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy as ps


C:\Users\Rodrigo\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Importando dados dos Games

In [3]:
# Esse é o local em que os dados estão salvos no meu Computador.
Games = pd.read_csv('C:\\Users\\Rodrigo\\Documents\\GitHub\\dsc_fev2018\\17-feature-engineering\\data.csv')

In [4]:
Games.head()


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


Fazer uma avaliação global sem considerar genero


Filtrando Jogos sem avaliação da crítica

In [5]:
# Esse é o filtro de jogos sem a avaliação da critica
Games2=Games[pd.notnull(Games['Critic_Score'])]
Games2.head(10)


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,Nintendo,14.44,6.94,4.70,2.24,28.32,87.0,80.0,8.4,594.0,Nintendo,E
11,Mario Kart DS,DS,2005.0,Racing,Nintendo,9.71,7.47,4.13,1.90,23.21,91.0,64.0,8.6,464.0,Nintendo,E
13,Wii Fit,Wii,2007.0,Sports,Nintendo,8.92,8.03,3.60,2.15,22.70,80.0,63.0,7.7,146.0,Nintendo,E
14,Kinect Adventures!,X360,2010.0,Misc,Microsoft Game Studios,15.00,4.89,0.24,1.69,21.81,61.0,45.0,6.3,106.0,Good Science Studio,E
15,Wii Fit Plus,Wii,2009.0,Sports,Nintendo,9.01,8.49,2.53,1.77,21.79,80.0,33.0,7.4,52.0,Nintendo,E


Filtrando jogos sem dados de vendas (se existirem)

In [6]:
# Esse é o filtro sem os dados de vendas globais.
# Vou partir daqui dado que essa é a variável resposta e não faz sentido fazer previsões para jogos que não temos esse dado
Games3=Games2[pd.notnull(Games2['Global_Sales'])]
Games3.head(10)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,Nintendo,14.44,6.94,4.70,2.24,28.32,87.0,80.0,8.4,594.0,Nintendo,E
11,Mario Kart DS,DS,2005.0,Racing,Nintendo,9.71,7.47,4.13,1.90,23.21,91.0,64.0,8.6,464.0,Nintendo,E
13,Wii Fit,Wii,2007.0,Sports,Nintendo,8.92,8.03,3.60,2.15,22.70,80.0,63.0,7.7,146.0,Nintendo,E
14,Kinect Adventures!,X360,2010.0,Misc,Microsoft Game Studios,15.00,4.89,0.24,1.69,21.81,61.0,45.0,6.3,106.0,Good Science Studio,E
15,Wii Fit Plus,Wii,2009.0,Sports,Nintendo,9.01,8.49,2.53,1.77,21.79,80.0,33.0,7.4,52.0,Nintendo,E


In [7]:
# Utilizei aqui a função criada pelo professor Luiz Henrique Outi Kauffmann para substituir os nulos e strings por np.ban
def troca(x):
    try:
        return float(x)
    except ValueError as err:
        return np.nan

In [8]:
# Começei aqui a fazer meu trabalho. Criei uma variável games_r para partir a análise

games_r = Games3

# Usei essa função para ter a lista de nomes de todas as colunas
cols_labels =  list(games_r.columns.values)
print(cols_labels)




['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count', 'Developer', 'Rating']


In [9]:
# Defini todas as colunas que eu gostaria de criar dummies
cols_dummy = ['Platform','Year_of_Release','Genre','Publisher','Developer','Rating']

#Apliquei a função criada pelo professor para trocar os strings do User_score para np.nan
games_r["User_Score"]=games_r["User_Score"].apply( troca ) 

#Usei a função get_dummies. Explicando os parâmetros:
# dummy_na = Cria uma dummy extra para os casos de NAN
#Columns = Passei aqui a lista com o nome das colunas que eu queria aplicar a função
#drop_first = Essa função exclui uma das dummies criadas para cada coluna. Exemplo: para caracterizar sexo masculino/feminino, seria criado apenas uma variável ( e não duas) 
games_rd = pd.get_dummies(games_r,dummy_na=True,columns=cols_dummy,drop_first=True)


In [10]:
# Retirei aqui todas as variáveis que não interessavam ara essa análise

#games_rd = games_rd.drop(['Name','NA_Sales','EU_Sales','JP_Sales', 'Other_Sales','User_Count','User_Score'],axis=1)
games_rd = games_rd.drop(['Name','NA_Sales','EU_Sales','JP_Sales', 'Other_Sales'],axis=1)

print(games_rd)


# ajustei o 'Critic Score' para a mesma escala do 'User_score'
games_rd['Critic_Score'] = games_rd['Critic_Score']/10




       Global_Sales  Critic_Score  Critic_Count  User_Score  User_Count  \
0             82.53          76.0          51.0         8.0       322.0   
2             35.52          82.0          73.0         8.3       709.0   
3             32.77          80.0          73.0         8.0       192.0   
6             29.80          89.0          65.0         8.5       431.0   
7             28.92          58.0          41.0         6.6       129.0   
8             28.32          87.0          80.0         8.4       594.0   
11            23.21          91.0          64.0         8.6       464.0   
13            22.70          80.0          63.0         7.7       146.0   
14            21.81          61.0          45.0         6.3       106.0   
15            21.79          80.0          33.0         7.4        52.0   
16            21.04          97.0          50.0         8.2      3994.0   
17            20.81          95.0          80.0         9.0      1588.0   
19            20.15      

In [11]:
# Aqui, simplismente queria visualizar a lista de variáveis do modelo criada pela get_dummies
cols_labels2 =  list(games_rd.columns.values)
cols_labels2 = pd.DataFrame(cols_labels2)


print(cols_labels2)

                                              0
0                                  Global_Sales
1                                  Critic_Score
2                                  Critic_Count
3                                    User_Score
4                                    User_Count
5                                   Platform_DC
6                                   Platform_DS
7                                  Platform_GBA
8                                   Platform_GC
9                                   Platform_PC
10                                  Platform_PS
11                                 Platform_PS2
12                                 Platform_PS3
13                                 Platform_PS4
14                                 Platform_PSP
15                                 Platform_PSV
16                                 Platform_Wii
17                                Platform_WiiU
18                                Platform_X360
19                                  Plat

In [12]:
#Tirando NAs de User Score
games_rd['User_Score'] = games_rd['User_Score'].apply(lambda x:0 if np.isnan(x)==True else x)
games_rd['User_Count'] = games_rd['User_Count'].apply(lambda x:0 if np.isnan(x)==True else x)

games_rd.head

<bound method NDFrame.head of        Global_Sales  Critic_Score  Critic_Count  User_Score  User_Count  \
0             82.53           7.6          51.0         8.0       322.0   
2             35.52           8.2          73.0         8.3       709.0   
3             32.77           8.0          73.0         8.0       192.0   
6             29.80           8.9          65.0         8.5       431.0   
7             28.92           5.8          41.0         6.6       129.0   
8             28.32           8.7          80.0         8.4       594.0   
11            23.21           9.1          64.0         8.6       464.0   
13            22.70           8.0          63.0         7.7       146.0   
14            21.81           6.1          45.0         6.3       106.0   
15            21.79           8.0          33.0         7.4        52.0   
16            21.04           9.7          50.0         8.2      3994.0   
17            20.81           9.5          80.0         9.0      1588.

In [13]:
#Separando o X_train e Y_train
X_train = games_rd.drop(['Global_Sales'],axis=1).values
Y_train = games_rd['Global_Sales'].values


In [14]:
games_rd.head

<bound method NDFrame.head of        Global_Sales  Critic_Score  Critic_Count  User_Score  User_Count  \
0             82.53           7.6          51.0         8.0       322.0   
2             35.52           8.2          73.0         8.3       709.0   
3             32.77           8.0          73.0         8.0       192.0   
6             29.80           8.9          65.0         8.5       431.0   
7             28.92           5.8          41.0         6.6       129.0   
8             28.32           8.7          80.0         8.4       594.0   
11            23.21           9.1          64.0         8.6       464.0   
13            22.70           8.0          63.0         7.7       146.0   
14            21.81           6.1          45.0         6.3       106.0   
15            21.79           8.0          33.0         7.4        52.0   
16            21.04           9.7          50.0         8.2      3994.0   
17            20.81           9.5          80.0         9.0      1588.

In [15]:
# Create linear regression object
regr = linear_model.LinearRegression()
#X_train=Y.reshape(len(Y),1)
Y_train=Y_train.reshape(len(Y_train),1)

# Train the model using the training sets
regr.fit(X_train, Y_train)

beta1=regr.coef_

intercepto=regr.intercept_

print(beta1)
print(intercepto)

#R quadrado 
regr.score(X_train,Y_train)

[[  1.72916249e-01   1.60790365e-02   1.58619947e-03 ...,  -4.54469753e+07
    5.66967431e+00   5.80392692e+00]]
[ -1.87908145e+09]


0.42092043208541308

In [16]:
#modelo com stat_models
model = sm.OLS(Y_train, X_train)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.421
Model:                            OLS   Adj. R-squared:                  0.259
Method:                 Least Squares   F-statistic:                     2.592
Date:                Fri, 30 Mar 2018   Prob (F-statistic):          1.59e-162
Time:                        09:50:54   Log-Likelihood:                -14181.
No. Observations:                8137   AIC:                         3.193e+04
Df Residuals:                    6354   BIC:                         4.442e+04
Df Model:                        1782                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1729      0.022      7.912      0.0